In [46]:
import tensorflow.keras.datasets as datasets
from random import sample
from matplotlib.pyplot import imshow
import matplotlib as plt
import numpy as np

In [47]:
#Parameters - To be defined by user
nClass = 10
nHidden = [256, 64]
nInput = 784
layers = [nInput] + nHidden + [nClass]
limit = 0.0001
#Hyperparameters - To be tuned by the user
learning_rate = 0.01
nMiniBatch = 128
nIter = 10000
print("Layers: ", layers)
print("Learning Rate: ", learning_rate)
print("Number of MiniBatch: ", nMiniBatch)
print("Iterations: ", nIter)

Layers:  [784, 256, 64, 10]
Learning Rate:  0.01
Number of MiniBatch:  128
Iterations:  10000


In [48]:
#-------data input function-----------------------------#
def getDataset(name, nClass):
    if name=="mnist":
        dataset = datasets.mnist
    (x_train, y_train),(x_test, y_test) = dataset.load_data()     #downloading and loading the dataset
    x_train, x_test = x_train / 255.0, x_test / 255.0             #normalizing the input data
    x_train_flat = x_train.reshape(x_train.shape[0],-1).T         #making dataset suitable for input in Fully Connected layer
    x_test_flat = x_test.reshape(x_test.shape[0],-1).T            #making dataset suitable for input in Fully Connected layer
    y_train_onehot = np.eye(nClass)[y_train].T                    #converting to one hot vectors
    y_test_onehot = np.eye(nClass)[y_test].T                      #converting to one hot vectors
    return x_train_flat,x_test_flat,y_train_onehot,y_test_onehot

In [49]:
x_train, x_test, y_train, y_test = getDataset("mnist", nClass)

In [59]:
class deepfuzzy:
    W = []
    b = []
    parameters = dict()
    act = []
    def __init__(self, layers, x_train, x_test, y_train, y_test, minibatch_size, load_weight = False learning_rate=0.01, iterations=100):
        self.layers = layers
        self.x_train_batch = x_train[:, :minibatch_size]
        self.x_test = x_test
        self.y_train_batch = y_train[:, :minibatch_size]
        self.y_test = y_test
        self.x_train = x_train
        self.y_train = y_train
        self.batch = minibatch_size
        self.iter = iterations
        self.learning_rate = learning_rate
        if(load_weight = False):
            self.initialize()
        else:
            self.loadWeights()
    def initialize(self, initializer = 'random'):
        if initiaizer = 'random':
            for i in range(len(self.layers)-1):
                self.W.append(np.random.rand(self.layers[i+1],self.layers[i])*0.02)
                self.b.append(np.random.rand(self.layers[i+1],1))
                assert(self.W[i].shape == (self.layers[i+1], self.layers[i]))
                assert(self.b[i].shape == (self.layers[i+1], 1))
        elif initializer == 'xavier':
            pass
        self.parameters['W'] = self.W
        self.parameters['b'] = self.b
    def forwardProp(self):
        self.act=[]
        self.act.append(self.x_train_batch)
        for i in range(len(self.layers)-2):
            z = np.dot(self.parameters['W'][i], self.act[-1])
            self.act.append(relu(z + self.parameters['b'][i]))  #relu
        self.act.append(softmax(np.dot(self.parameters['W'][len(self.layers)-2], self.act[-1])))
    def compCost(self):
        interm = np.dot(np.log(self.act[-1]).T,self.y_train_batch)
        cost = -1.0/self.y_train_batch.shape[1]*np.sum(np.trace(interm))
        return cost
    def backProp(self):
        m = self.y_train_batch.shape[1]
        dZ = self.act[-1] - self.y_train_batch
        dW = 1.0/m * np.dot(dZ, self.act[-2].T)
        db = 1.0/m * np.dot(dZ, self.act[-2].T)
        dA_prev = np.dot(self.parameters['W'][-1].T, dZ)
        self.parameters['W'][-1] = self.parameters['W'][-1] - self.learning_rate*dW
        self.parameters['b'][-1] = self.parameters['b'][-1] - self.learning_rate*db
        for i in reversed(range(len(self.layers)-2)):
            dA = dA_prev 
            dZ = linear_activation_backward(dA,self.act[i+1],"relu")
            dW = 1.0/m * np.dot(dZ, self.act[i].T)
            db = 1.0/m*  np.sum(np.array(dZ),axis=1,keepdims=True)
            dA_prev = np.dot(self.parameters['W'][i].T, dZ)
            self.parameters['W'][i] = self.parameters['W'][i] - self.learning_rate*dW
            self.parameters['b'][i] = self.parameters['b'][i] - self.learning_rate*db
    def check_accuracy(self, y, x):
        mat = np.zeros([10,10])
        m = y.shape[1]
        buff_x = self.x_train_batch
        self.x_train_batch = x[:,]
        self.forwardProp()
        pred = np.argmax(self.act[-1], axis = 0)
        exp = np.argmax(y, axis = 0)
        error = np.sum(exp!=pred)
        self.x_train_batch = buff_x
        for i in range(m):
            mat[exp[i]][pred[i]] =  mat[exp[i]][pred[i]] + 1
        # Calculate accuracy
        return (m - error)/m * 100, mat
    def train(self):
        for i in range(self.iter):
                if(i%2==0):
                    idx = np.random.randint(self.x_train.shape[1], size=self.batch)
                    self.x_train_batch = self.x_train[:,idx]
                    self.y_train_batch = self.y_train[:,idx]
                self.forwardProp()
                cost = self.compCost()
                self.backProp()
                if(i%100 == 0):
                    Accuracy, _ = self.check_accuracy(y_train,x_train)
                    print("Accuracy: ", Accuracy)
                    self.saveWeights()
    def test(self):
        self.forwardProp()
        Accuracy, mat = self.check_accuracy(y_test,x_test)
        print("Test Accuracy", Accuracy )
        print("Confusion Matrix:")
        print(mat)
    def saveWeights(self):
        np.save('W',self.parameters['W'])
        np.save('b',self.parameters['b'])
    def loadWeights(self):
        try:
            W = np.load('W.npy')
            b = np.load('b.npy')
            self.parameters['W'] = W
            self.parameters['b'] = b
        except:
            print("Not able to load weights!")
            print("Initializing Weights...")
            self.initialize()
            print("Done!")

In [60]:
test = deepfuzzy(layers, x_train, x_test, y_train, y_test, nMiniBatch, learning_rate=0.01, iterations=10000)

In [61]:
test.initialize()

In [62]:
test.train()

Accuracy:  9.863333333333333
Accuracy:  9.863333333333333
Accuracy:  10.218333333333334
Accuracy:  10.218333333333334


KeyboardInterrupt: 

In [63]:
test.test()

Test Accuracy 13.5
